In [1]:
# ещё один скрипт для скользящего отслеживания рынков, он так же связан с fast_saver.py

In [1]:
import pandas as pd
import numpy as np
import os

from bittrex import Bittrex
bittrex = Bittrex(None, None)

In [2]:
%cd fast_dump/

/home/me/soft/other/python-bittrex-master/bittrex/fast_dump


In [3]:
# функция сравнения - сравнивает два снимка рынка, ищет разницу объема или цены в разные промежутки времени
# window_size - индекс из списков edge, параметр окна
def cmp(recent, early, window_size):
    recent = pd.read_csv(recent)     # получить ранний снимок
    early  = pd.read_csv(early)      # получить поздний снимок
    
    edge_v = [1.05, 1.05, 1.12, 1.2] # сигналы по объему: 5% за 5 м, 5% за 15 м, 12% за 30м, 20% за 60м
    edge_l = [1.04, 1.05, 1.1, 1.15] # сигналы по цене:   4% за 5 м, 5% за 15 м, 10% за 30м, 15% за 60м
    
    names = recent['MarketName']                            # получить имя маркета
    vol = recent['BaseVolume'] / early['BaseVolume']        # получить изменения объема
    last = recent['Last'] / early['Last']                   # получить изменение цены
    vol = np.array(vol)
    last = np.array(last)
    vol_ind = np.where(vol >= edge_v[window_size])          # поиск маркетов, подходящих под требование
    last_ind = np.where(last >= edge_l[window_size])        #
    
    indx = last_ind + vol_ind

    if np.shape(indx) == (2, 0): # отлов ошибок
        return -1
    
    names_last = []
    names_vol =  []
    
    for i in last_ind:
        names_last.append(names[i])
        
    for i in vol_ind:
        names_vol.append(names[i])
        
    return (names_last, names_vol)

In [4]:
def mov_window(ld, time):   # time - размер окна, и, соответственно, пороговые значения для срабатывания

    result = cmp (ld[-1], ld[-5-1], time)
    if type(result) != int:
        print (result)


In [7]:
ld = os.listdir()
ld.sort()
del ld[-5:]
mov_window(ld, 1)

([29      BTC-BYC
68      BTC-FTC
121     BTC-PDC
230    ETH-PTOY
Name: MarketName, dtype: object], [26    BTC-BTCD
29     BTC-BYC
Name: MarketName, dtype: object])
